## Cleanup

#### Imports 

In [ ]:
from google.cloud.exceptions import NotFound
from google.cloud import bigquery
import pandas as pd
import logging
import os 

##### Setup logging

In [ ]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

#### Setup essentials

In [ ]:
SERVICE_ACCOUNT_CREDENTIALS = './../credentials/vai-key.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = SERVICE_ACCOUNT_CREDENTIALS

In [ ]:
client = bigquery.Client()

In [ ]:
dataset_id = f"{client.project}.flight_reservations"
logger.info(dataset_id)

#### Delete all tables in the dataset

In [ ]:
dataset_ref = bigquery.DatasetReference.from_string(dataset_id)
tables = client.list_tables(dataset_ref)
for table in tables:
    table_ref = dataset_ref.table(table.table_id)
    try:
        client.delete_table(table_ref) 
        logger.info(f"Deleted table '{table.table_id}' from dataset '{dataset_id}'.")
    except NotFound:
        logger.warn(f"Table '{table.table_id}' not found in dataset '{dataset_id}'.")


#### Delete the dataset

In [ ]:
try:
    client.delete_dataset(dataset_id, delete_contents=True, not_found_ok=True)  
    logger.info(f"Deleted dataset '{dataset_id}'.")
except NotFound:
    logger.warn(f"Dataset '{dataset_id}' not found.")